# Data prep/HDI indicators

In [4]:
import requests
import json
import pandas as pd
import os
os.chdir("..")

In [ ]:
API_KEY = ""

In [3]:
url = f"https://hdrdata.org/api/CompositeIndices/query?apikey={API_KEY}&countryOrAggregation=AFG&year=2022"

In [4]:
response = requests.get(url)

In [5]:
response.status_code

200

In [6]:
url = f"https://hdrdata.org/api/Metadata/Indicators?apikey={API_KEY}"

In [7]:
response = requests.get(url)

In [8]:
input_ = [(output["code"],output["name"]) for output in response.json()]

In [9]:
pd.DataFrame(input_, columns=["code", "name"]).to_excel("HDI_indicators.xlsx", index=False)

eys	: Expected Years of Schooling (years) \
hdi : Human Development Index (value) \
le	: Life Expectancy at Birth (years) \
mys	: Mean Years of Schooling (years) \
mmr :	Maternal Mortality Ratio (deaths per 100,000 live births) \
child_mortality	: Child mortality (%) \
gnipc : Gross National Income Per Capita (2017 PPP$)

In [10]:
indicators = {
"eys":"Expected Years of Schooling (years)",
"hdi":"Human Development Index (value)",
"le":"Life Expectancy at Birth (years)",
"mys":"Mean Years of Schooling (years)",
"mmr":"Maternal Mortality Ratio (deaths per 100,000 live births)",
"child_mortality":"Child mortality (%)",
"gnipc":"Gross National Income Per Capita (2017 PPP$)"
}

In [11]:
ind_str = ",".join([ind for ind in indicators.keys()])

In [12]:
query = f"https://hdrdata.org/api/CompositeIndices/query-detailed?apikey={API_KEY}&year=2015&indicator={ind_str}"

In [13]:
response = requests.get(query)

In [15]:
country_data = {}
for data_output in response.json():
    isoCode = data_output["countryIsoCode"]
    indCode = data_output["indicatorCode"]
    if isoCode not in country_data.keys():
        country_data[isoCode] = {}
    country_data[isoCode][indCode] = data_output["value"]

In [16]:
hdi_data = pd.DataFrame.from_dict(country_data, orient="index")

In [ ]:
emb = pd.read_excel("embeddings/all_nw500_1000_wl80_vs75_pca4.xlsx")
df_emb = emb[["Node","Embedding"]].set_index("Node", drop=True)
df_merge =df_emb.merge(hdi_data, left_index=True,right_index=True, how="left")

In [22]:
df_merge.groupby("Embedding").mean()

,child_mortality,eys,gnipc,hdi,le,mmr,mys
Embedding,,,,,,,
0,11.9126,12.678641,15975.186094,0.689141,70.704108,168.926492,8.149031
1,6.9965,12.885268,13899.585707,0.676122,68.664878,206.297600,8.213220
2,49.7610,15.545977,33365.807047,0.832628,77.677773,29.143857,10.902488
3,5.9090,12.451167,15461.260674,0.668214,69.649512,226.191024,7.416119


In [26]:
df_merge.to_excel("metadata_country/metadata_country.xlsx")

In [8]:
df_3 = pd.read_excel("metadata_country/metadata_country.xlsx").set_index("Node",drop=True)

In [12]:
df_3["Embedding_4"] = df_3["Embedding"]
df_3 = df_3.drop("Embedding", axis=1)

In [13]:
emb_3 = pd.read_excel("embeddings/all_nw500_1000_wl80_vs75_pca3.xlsx")
df_emb_3 = emb_3[["Node","Embedding"]].set_index("Node", drop=True)
df_emb_3["Embedding_3"] = df_emb_3["Embedding"]
df_emb_3 = df_emb_3.drop("Embedding", axis=1) 
df_merge_3 =df_emb_3.merge(df_3, left_index=True,right_index=True, how="left")

In [14]:
df_merge_3.groupby("Embedding_3").mean()

,child_mortality,eys,gnipc,hdi,le,mmr,mys,Embedding_4
Embedding_3,,,,,,,,
0,25.8825,15.232000,30861.525600,0.821686,76.830514,37.582200,10.749357,1.642857
1,1.9010,12.754381,13264.792098,0.672805,69.618357,183.256235,8.030732,1.054795
2,5.9408,11.959203,12346.194600,0.640392,68.168778,252.889538,7.048722,1.253012


In [15]:
df_merge_3.to_excel("metadata_country/metadata_country_3.xlsx")